In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                #running_loss += loss.data[0]
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

if __name__ == '__main__':

    # data_transform, pay attention that the input of Normalize() is Tensor and the input of RandomResizedCrop() or RandomHorizontalFlip() is PIL Image
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomSizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    # your image data file
    #data_dir = 'E:\design\data'
    image_datasets = {x: datasets.ImageFolder('E:\design\data',
                                              data_transforms[x]) for x in ['train', 'val']}
    # wrap your data and label into Tensor
    dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=4,
                                                 shuffle=True,
                                                 num_workers=4) for x in ['train', 'val']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    # use gpu or not
    use_gpu = torch.cuda.is_available()

    # get model and replace the original fc layer with your fc layer
    model_ft = models.resnet18(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, 2)

    if use_gpu:
        model_ft = model_ft.cuda()

    # define loss function
    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

    model_ft = train_model(model=model_ft,
                           criterion=criterion,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=25)

D:\Anacoda\lib\site-packages\torchvision\transforms\transforms.py:618: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +
D:\Anacoda\lib\site-packages\torchvision\transforms\transforms.py:207: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


Epoch 0/24
----------
train Loss: 0.1532 Acc: 0.7244
val Loss: 0.0836 Acc: 0.8409
Epoch 1/24
----------
train Loss: 0.1292 Acc: 0.7786
val Loss: 0.0997 Acc: 0.8293
Epoch 2/24
----------
train Loss: 0.1200 Acc: 0.7871
val Loss: 0.0689 Acc: 0.8855
Epoch 3/24
----------
train Loss: 0.1181 Acc: 0.7967
val Loss: 0.0718 Acc: 0.8524
Epoch 4/24
----------
train Loss: 0.1116 Acc: 0.8012
val Loss: 0.0780 Acc: 0.8409
Epoch 5/24
----------
train Loss: 0.1111 Acc: 0.8218
val Loss: 0.0596 Acc: 0.8735
Epoch 6/24
----------
train Loss: 0.1014 Acc: 0.8243
val Loss: 0.0522 Acc: 0.9066
Epoch 7/24
----------
train Loss: 0.0899 Acc: 0.8424
val Loss: 0.0522 Acc: 0.9086
Epoch 8/24
----------
train Loss: 0.0825 Acc: 0.8594
val Loss: 0.0505 Acc: 0.9006
Epoch 9/24
----------
train Loss: 0.0882 Acc: 0.8529
val Loss: 0.0466 Acc: 0.9217
Epoch 10/24
----------
train Loss: 0.0792 Acc: 0.8539
val Loss: 0.0429 Acc: 0.9282
Epoch 11/24
----------
train Loss: 0.0780 Acc: 0.8680
val Loss: 0.0428 Acc: 0.9232
Epoch 12/24
--

In [2]:
torch.save(model_ft, 'model2.pkl')
#model = torch.load('model.pkl')

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os
model = torch.load('model2.pkl')

In [2]:
#model.eval()

In [5]:
data_transforms = transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    
# your image data file
    #data_dir = 'E:\design\data'
image_datasets = datasets.ImageFolder('E:\design\test_data')
#image_datasets = data_transforms[image_datasets]
    # wrap your data and label into Tensor
dataloders = torch.utils.data.DataLoader(image_datasets,
                                                 batch_size=4,
                                                 shuffle=True,
                                                 num_workers=4)
#dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
y=model(image_datasets)
print(y)

OSError: [WinError 123] 文件名、目录名或卷标语法不正确。: 'E:\\design\test_data'

In [10]:
data_transforms =transforms.Compose([
            transforms.Scale(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
from PIL import Image
im = Image.open('ers.jpg')
print(im)
image_data=data_transforms(im)
image_data=image_data.unsqueeze(dim=0)
image_data=image_data.cuda()#把image_data放入GPU
#image_data2 = torch.utils.data.DataLoader(image_data,batch_size=4,shuffle=True,num_workers=4) 


y=model(image_data)
print(y)
y=y.cpu()#把y放回cpu
print(y.data.numpy())

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3840x2160 at 0x1F00A48BE80>
tensor([[ 1.3706, -1.1921]], device='cuda:0', grad_fn=<AddmmBackward>)
[[ 1.3705509 -1.1920785]]


In [15]:
w=y.data.numpy()
print(w)
if w[0,0]>0:
    label='NO'
else:
    lavber='Yes'
print(label)

[[ 1.3705509 -1.1920785]]
NO
